# download test images

In [1]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1GL_Rh1N-WjrvF_-YOKOyvq0zrV6TF4hb',
                                    dest_path='/content/dataset.zip',
                                    unzip=True)

Unzipping...Done.


# download the weight

In [2]:
gdd.download_file_from_google_drive(file_id='162is67vW9CYdYyh9vk8W76UMfmORygCS',
                                    dest_path='/content/edsr_350000.h5',
                                    unzip=False)

# EDSR model

In [14]:
import os
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.python.data.experimental import AUTOTUNE
import numpy as np
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Add, Conv2D, Input, Lambda
DIV2K_RGB_MEAN = np.array([0.4488, 0.4371, 0.4040]) * 255
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.metrics import Mean
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
tf.keras.mixed_precision.experimental.set_policy(policy) 
import time
import os

def normalize(x,rgb_mean=DIV2K_RGB_MEAN):
    '''This function will normalize image by substracting RGB mean from image'''
    return (x - rgb_mean) / 127.5

def denormalize(x,rgb_mean=DIV2K_RGB_MEAN):
    ''' This function will denormalize image by adding back rgb_mean'''
    return (x * 127.5 )+ rgb_mean

def shuffle_pixels(scale):
    return lambda x: tf.nn.depth_to_space(x, scale)

def ResBlock(x_input, num_filters, scaling):
    '''Thpis function Implementes Proposed ResBlock Architecture as per EDSR paper'''
    # proposed ResBlock ==> Conv --> Relu --> Conv --> Scaling(mul) --> Add
    x = Conv2D(num_filters, 3, padding='same', activation='relu')(x_input)
    x = Conv2D(num_filters, 3, padding='same')(x)
    if scaling:
        x = Lambda(lambda t: t * scaling)(x)
    x = Add()([x_input, x])
    return x

def Upsampling(x, scale, num_filters):
    '''This function upsampling as mentioned in EDSR paper'''
    def upsample(x, factor, **kwargs):
        x = Conv2D(num_filters * (factor ** 2), 3, padding='same', **kwargs)(x)
        return Lambda(shuffle_pixels(scale=factor))(x)

    if scale == 2:
        x = upsample(x, 2, name='conv2d_1_scale_2')
    elif scale == 3:
        x = upsample(x, 3, name='conv2d_1_scale_3')
    elif scale == 4:
        x = upsample(x, 2, name='conv2d_1_scale_2')
        x = upsample(x, 2, name='conv2d_2_scale_2')

    return x


def EDSR(scale, num_filters=64, res_blocks=8, res_block_scaling=None):
    x_input = Input(shape=(None, None, 3))
    # Normalize input with DIV2K_RGB_MEAN
    x = Lambda(normalize)(x_input)
    
    # assign value of x to x_res block for further operations
    x = x_res_block = Conv2D(num_filters, 3, padding='same')(x)

    # Goes in number of res block
    for i in range(res_blocks):
        x_res_block = ResBlock(x_res_block, num_filters, res_block_scaling)
    # convolution
    x_res_block = Conv2D(num_filters, 3, padding='same')(x_res_block)
    # add res_block output and original normalizwd input
    x = Add()([x, x_res_block])

    # upsampling
    x = Upsampling(x, scale, num_filters)
    x = Conv2D(3, 3, padding='same')(x)
    
    # denormalize to get back original form
    x = Lambda(denormalize)(x)
    return Model(x_input, x, name="EDSR")


Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  Tesla P100-PCIE-16GB, compute capability 6.0
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


In [15]:
new_model = EDSR(scale=3,res_blocks=16)
new_model.load_weights('/content/edsr_350000.h5')

# reproduce the result

In [12]:
os.mkdir('/content/result')

In [17]:
import cv2

test_dir = '/content/testing_lr_images/testing_lr_images/'
for file_name in os.listdir(test_dir):
  print(file_name)
  path = test_dir + file_name
  img = cv2.imread(path)

  img = tf.expand_dims(img,axis=0)
  img = tf.cast(img, tf.float32)
  sr_img = new_model(img)
  sr_img = tf.clip_by_value(sr_img, 0, 255)
  sr_img = tf.round(sr_img)
  sr_img = tf.cast(sr_img, tf.uint8)
  result = np.reshape(sr_img.numpy(), (sr_img.numpy().shape[1], sr_img.numpy().shape[2],3))

  des = '/content/result/'+file_name[:-4]+'_pred.png'
  cv2.imwrite(des,result)

04.png
11.png
13.png
00.png
07.png
09.png
05.png
08.png
02.png
10.png
12.png
03.png
06.png
01.png
